<a href="https://colab.research.google.com/github/harveenchadha/bol/blob/main/demos/gr/hindi/Speech_Translation_Hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install gradio
!pip install transformers torchaudio
!apt-get install sox
!pip install sentencepiece

In [2]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer


def translate(text):
    model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
    tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
    inlang='hi'
    outlang='en'
    tokenizer.src_lang = inlang
    encoded_hi = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id(outlang))
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]


In [3]:
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import argparse
from glob import glob
import torchaudio
import subprocess
import gradio as gr

resampler = torchaudio.transforms.Resample(48_000, 16_000)

def get_filename(wav_file):
    filename_local = wav_file.split('/')[-1][:-4]
    filename_new = '/tmp/'+filename_local+'_16.wav'
    
    
    subprocess.call(["sox {} -r {} -b 16 -c 1 {}".format(wav_file, str(16000), filename_new)], shell=True)
    return filename_new

def parse_transcription(wav_file):
    # load pretrained model
    processor = Wav2Vec2Processor.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")
    model = Wav2Vec2ForCTC.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")

    # load audio

    
    wav_file = get_filename(wav_file.name)
    audio_input, sample_rate = sf.read(wav_file)
    #test_file = resampler(test_file[0])

    # pad input values and return pt tensor
    input_values = processor(audio_input, sampling_rate=16_000, return_tensors="pt").input_values

    # INFERENCE
    # retrieve logits & take argmax
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    # transcribe
    transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)
    return translate(transcription)
    #return transcription

In [ ]:
input = gr.inputs.Audio(source="microphone", type="file") 
# io1 = gr.Interface(parse_transcription, inputs = input,  outputs="text", analytics_enabled=False, show_tips=False, theme='huggingface')
# io2 = gr.Interface(translate, inputs = "text", outputs="text", theme='huggingface')

gr.Interface(parse_transcription, inputs = input,  outputs="text", analytics_enabled=False, show_tips=False, theme='huggingface').launch(debug=True, inline=False)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Your interface requires microphone or webcam permissions - this may cause issues in Colab. Use the External URL in case of issues.
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://41783.gradio.app


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
